In [22]:
import cudf

import numpy as np
import pandas as pd
from tqdm import tqdm
import datetime
from collections import Counter

# Forming Train Set 🔹

We'll keep 2 weeks as train and the last week as validation.

In [23]:

data = df.to_pandas() 
data["t_dat"] = pd.to_datetime(data["t_dat"])
data.head()

,t_dat,customer_id,article_id
0,2018-09-20,-6846340800584936,663713001
1,2018-09-20,-6846340800584936,541518023
2,2018-09-20,-8334631767138808638,505221004
3,2018-09-20,-8334631767138808638,685687003
4,2018-09-20,-8334631767138808638,685687004


In [24]:

print("All Transactions Date Range: {} to {}".format(data['t_dat'].min(), data['t_dat'].max()))

data["t_dat"] = pd.to_datetime(data["t_dat"])
train1 = data.loc[(data["t_dat"] >= datetime.datetime(2020,9,8)) & (data['t_dat'] < datetime.datetime(2020,9,16))]
train2 = data.loc[(data["t_dat"] >= datetime.datetime(2020,9,1)) & (data['t_dat'] < datetime.datetime(2020,9,8))]
train3 = data.loc[(data["t_dat"] >= datetime.datetime(2020,8,23)) & (data['t_dat'] < datetime.datetime(2020,9,1))]
train4 = data.loc[(data["t_dat"] >= datetime.datetime(2020,8,15)) & (data['t_dat'] < datetime.datetime(2020,8,23))]

val = data.loc[data["t_dat"] >= datetime.datetime(2020,9,16)]

All Transactions Date Range: 2018-09-20 00:00:00 to 2020-09-22 00:00:00


In [25]:

positive_items_per_user1 = train1.groupby(['customer_id'])['article_id'].apply(list)
positive_items_per_user2 = train2.groupby(['customer_id'])['article_id'].apply(list)
positive_items_per_user3 = train3.groupby(['customer_id'])['article_id'].apply(list)
positive_items_per_user4 = train4.groupby(['customer_id'])['article_id'].apply(list)

In [26]:
train = pd.concat([train1, train2, train3, train4], axis=0)

#time decay popularity of each article
train['pop_factor'] = train['t_dat'].apply(lambda x: 1/(datetime.datetime(2020,9,16) - x).days**2)
popular_items_group = train.groupby(['article_id'])['pop_factor'].sum()

# purchase count of each article
items_total_count = train.groupby(['article_id'])['article_id'].count()
# purchase count of each user
users_total_count = train.groupby(['customer_id'])['customer_id'].count()


train['feedback'] = 1

In [27]:
train = train.drop(['t_dat'], axis=1)

In [29]:

train = train.groupby(['customer_id', 'article_id']).sum().reset_index()
train['feedback'] = train.apply(lambda row: row['feedback']/popular_items_group[row['article_id']], axis=1)

train['feedback'] = train['feedback'].apply(lambda x: 5.0 if x>5.0 else x)


# shuffling
train = train.sample(frac=1).reset_index(drop=True)
train['feedback'].describe()

count    1.044811e+06
mean     8.409321e-01
std      1.692177e+00
min      2.887584e-05
25%      3.906699e-03
50%      2.891109e-02
75%      3.873003e-01
max      5.000000e+00
Name: feedback, dtype: float64

In [30]:
train_pop = data.loc[(data["t_dat"] >= datetime.datetime(2020,9,1)) & (data['t_dat'] < datetime.datetime(2020,9,16))]
train_pop['pop_factor'] = train_pop['t_dat'].apply(lambda x: 1/(datetime.datetime(2020,9,16) - x).days)
popular_items_group = train_pop.groupby(['article_id'])['pop_factor'].sum()

_, popular_items = zip(*sorted(zip(popular_items_group, popular_items_group.keys()))[::-1])

train_pop['pop_factor'].describe()

/tmp/ipykernel_2226670/2393265638.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_pop['pop_factor'] = train_pop['t_dat'].apply(lambda x: 1/(datetime.datetime(2020,9,16) - x).days)


count    557958.000000
mean          0.200478
std           0.207752
min           0.066667
25%           0.083333
50%           0.125000
75%           0.200000
max           1.000000
Name: pop_factor, dtype: float64

Below part is a cool new addition which gives a small bump. Did not add it to the validation part but it has the same effect there too. It predicts the most frequent next item bought by an user after a particular item.

In [31]:
def get_most_freq_next_item(user_group):
    next_items = {}
    for user in tqdm(user_group.keys()):
        items = user_group[user]
        for i,item in enumerate(items[:-1]):
            if item not in next_items:
                next_items[item] = []
            if item != items[i+1]:
                next_items[item].append(items[i+1])

    pred_next = {}
    for item in next_items:
        if len(next_items[item]) >= 5:
            most_common = Counter(next_items[item]).most_common()
            ratio = most_common[0][1]/len(next_items[item])
            if ratio >= 0.1:
                pred_next[item] = most_common[0][0]
            
    return pred_next

user_group = train.groupby(['customer_id'])['article_id'].apply(list)
pred_next = get_most_freq_next_item(user_group)

100%|██████████| 254618/254618 [00:04<00:00, 51020.34it/s]


In [32]:
def apk(actual, predicted, k=12):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=12):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [33]:
positive_items_val = val.groupby(['customer_id'])['article_id'].apply(list)
val_users = positive_items_val.keys()
val_items = []

for i,user in tqdm(enumerate(val_users)):
    val_items.append(positive_items_val[user])
    
print("Total users in validation:", len(val_users))

68984it [00:01, 68437.10it/s]

Total users in validation: 68984


#### Normal way of prediction without the SVD reranking

In [34]:
from collections import Counter
outputs = []
cnt = 0

popular_items = list(popular_items)

for user in tqdm(val_users):
    user_output = []
    if user in positive_items_per_user1.keys():
        most_common_items_of_user = {k:v for k, v in Counter(positive_items_per_user1[user]).most_common()}
        user_output += list(most_common_items_of_user.keys())[:12]
    if user in positive_items_per_user2.keys():
        most_common_items_of_user = {k:v for k, v in Counter(positive_items_per_user2[user]).most_common()}
        user_output += list(most_common_items_of_user.keys())[:12]
    if user in positive_items_per_user3.keys():
        most_common_items_of_user = {k:v for k, v in Counter(positive_items_per_user3[user]).most_common()}
        user_output += list(most_common_items_of_user.keys())[:12]
    if user in positive_items_per_user4.keys():
        most_common_items_of_user = {k:v for k, v in Counter(positive_items_per_user4[user]).most_common()}
        user_output += list(most_common_items_of_user.keys())[:12]
    
    user_output += [pred_next[item] for item in user_output if item in pred_next and pred_next[item] not in user_output]      
    
    user_output += list(popular_items[:12 - len(user_output)])
    outputs.append(user_output)
    
print("mAP Score on Validation set:", mapk(val_items, outputs))

100%|██████████| 68984/68984 [00:08<00:00, 7867.91it/s]


mAP Score on Validation set: 0.024382187988015183
